# A Grid Class and Ghost Cells

Consider comstructing a derivate using a difference approximation of the form:

$$\frac{\partial a}{\partial x} \approx \frac{a_{i+1} - a_{i-1}}{2 \Delta x}$$

Now imagine that our grid has $N$ zones, indexed by $i = 0, 1, \ldots, N-1$.  Our difference approximation for $\partial a/\partial x |_i=0$ is:

$$\left .\frac{\partial a}{\partial x} \right |_{i=0} \approx \frac{a_1 - a_{-1}}{2\Delta x}$$

But we don't have a value of $a$ corresponding to $i=-1$.  

This is where our boundary conditions come into play, and to accomodate our boundary conditions and allow ourselves to use the same difference
approximation everywhere in the domain, we extend the grid using *ghost cells*.

![ghost cells in the finite-volume method](fv_ghost.png)

In the figure above, the first zone inside the domain is indexed using `lo` and the last zone in the domain is indexed with `hi`.  Outside of the domain, there are 2 ghost cells on each end---we will initialize the data in these cells using our knowledge of the boundary conditions.

We need a class that holds the data on a finite-volume grid, including ghost cells.  Here's our implementation:

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class FVGrid:

    def __init__(self, nx, ng=2, xmin=0.0, xmax=1.0):

        self.xmin = xmin
        self.xmax = xmax
        self.ng = ng
        self.nx = nx

        # python is zero-based.  Make easy integers to know where the
        # real data lives
        self.ilo = ng
        self.ihi = ng+nx-1

        # physical coords -- cell-centered, left and right edges
        self.dx = (xmax - xmin)/(nx)
        self.x = xmin + (np.arange(nx+2*ng)-ng+0.5)*self.dx
        self.xl = xmin + (np.arange(nx+2*ng)-ng)*self.dx
        self.xr = xmin + (np.arange(nx+2*ng)-ng+1.0)*self.dx

        # storage for the solution
        self.a = self.scratch_array()
        self.ainit = self.scratch_array()

    def period(self, u):
        """ return the period for advection with velocity u """
        return (self.xmax - self.xmin) / np.abs(u)

    def scratch_array(self):
        """ return a scratch array dimensioned for our grid """
        return np.zeros((self.nx+2*self.ng), dtype=np.float64)

    def fill_BCs(self, atmp):
        """ fill all ghostcells with periodic boundary conditions """

        # left boundary
        for n in range(self.ng):
            atmp[self.ilo-1-n] = atmp[self.ihi-n]

        # right boundary
        for n in range(self.ng):
            atmp[self.ihi+1+n] = atmp[self.ilo+n]

    def norm(self, e):
        """ return the norm of quantity e which lives on the grid """
        if not len(e) == (2*self.ng + self.nx):
            return None

        return np.sqrt(self.dx*np.sum(e[self.ilo:self.ihi+1]**2))
    
    def plot(self):
        fig = plt.figure()
        ax = fig.add_subplot(111)

        ax.plot(self.x, self.ainit, label="initial conditions")
        ax.plot(self.x, self.a)
        ax.legend()
        return fig

## Interface values

We need to store interface values (which we've been denoting with half-integer subscripts, like $a_{i-1/2}$) as arrays which only take integer indices.  We will adopt the common convention that when an array refers to an interface state, it denotes the left interface of a cell.

E.g., `aint[i]}` means $a_{i-1/2}$

This is illustrated below for arrays `a[]` which stores cell center quantities and `aint[]` which stores interface values:

![array notation for interfaces](array-labels.png)